# __Long Short Term Memory Decoder__

### __Deep Learning__

#### __Project: Image Captioning with Visual Attention__

In [1]:
import os
os.chdir(os.environ["PYTHONPATH"])

import torch
import matplotlib.pyplot as plt
import seaborn as sns

import scripts.data_loading as dl
import scripts.data_processing as dp
from scripts import model

%matplotlib inline
%load_ext autoreload
%autoreload 2


plt.rcParams["figure.figsize"] = (10, 10)
plt.rcParams["image.cmap"] = "plasma"

In [2]:
coco_train = dl.CocoCaptions(
    dl.DATASET_PATHS[dl.DatasetType.TRAIN],
    dp.VGGNET_PREPROCESSING_PIPELINE,
    dp.TextPipeline(),
)

loading annotations into memory...
Done (t=2.01s)
creating index...
index created!


In [3]:
coco_loader = dl.CocoLoader(coco_train, batch_size=2, num_workers=1)
it = iter(coco_loader)
image_batch, caption_batch = next(it)

In [4]:
encoder = model.VGG19Encoder()
feature_maps, feature_mean = encoder.forward(image_batch)

In [5]:
decoder = model.LSTMDecoder(
    num_embeddings=len(coco_train.target_transform.vocabulary),
    embedding_dim=8,
    encoder_dim=feature_mean.shape[-1],
    decoder_dim=16,
    attention_dim=4
)

In [6]:
embeddings = decoder.word_embedding(caption_batch)

In [7]:
print(f"One-hot encoded caption shape = {caption_batch[0].shape}")
print(f"Embedding shape = {embeddings[0].shape}")

One-hot encoded caption shape = torch.Size([51])
Embedding shape = torch.Size([51, 8])


In [8]:
print(caption_batch[0])
print(embeddings[0])

tensor([10000,    12,    78,    22,    34,   850,    31,     4,     0,   182,
            1,  1421,    65,     2,     0,     8,   283,  1486,     0,    34,
          204,     9,  1584,    47,    34,   924,     1,    34,  1118,     5,
          133,   167,    62,    34,  1186,     4,   121,     4,    36,     2,
          244,     6,    12,  1186,     1,   206,     6,   314, 10002,   139,
        10001])
tensor([[ 1.9134,  0.9848, -0.7246,  0.1570,  0.1824, -1.0552, -0.4120, -0.6387],
        [ 0.5580, -0.8628,  0.5702,  0.1247, -0.7439, -0.3852, -0.7410,  0.1005],
        [ 0.8604, -0.6530, -0.0487,  0.2985,  0.6859, -1.8604,  0.5384, -0.5917],
        [-0.5398,  1.3453, -1.3022, -0.9039, -0.6064, -0.9585, -0.1857,  1.4214],
        [-0.5563,  0.7905, -0.6966,  1.5633, -0.3398, -0.9300,  1.4303, -0.8875],
        [-0.4930,  0.1075,  1.0731,  0.3416,  0.3918, -1.4074,  0.6666, -1.4724],
        [ 0.5533, -0.0096,  0.1748, -1.9221, -0.3489,  1.6190, -0.9427,  0.3558],
        [ 0.9285,  1

In [9]:
h = decoder.init_h(feature_mean)
c = decoder.init_c(feature_mean)

print(f"Initial h shape = {h.shape}")
print(f"Initial c shape = {c.shape}")

Initial h shape = torch.Size([2, 16])
Initial c shape = torch.Size([2, 16])


In [10]:
# Initial h, c of LSTM computed by MLP(feature_maps_mean)
print(h[0])
print(c[0])

tensor([ 0.2317,  0.1272,  0.1105, -0.0560,  0.1991, -0.1430,  0.0248,  0.0135,
        -0.0825,  0.2156,  0.0903,  0.0265,  0.2117, -0.1362,  0.0697,  0.0926],
       grad_fn=<SelectBackward>)
tensor([-0.0881, -0.0270, -0.0870, -0.1154, -0.1455,  0.1070, -0.3064,  0.3065,
         0.1521, -0.0409, -0.0123,  0.0334, -0.3113, -0.1511,  0.0573,  0.4214],
       grad_fn=<SelectBackward>)


In [11]:
# How to get word embeddings of words at particular index of a caption in batch
index = 50
embeddings[:, index]

tensor([[-0.6427, -0.5332, -1.0963,  0.3655,  1.5181, -0.4637, -1.3757,  0.6313],
        [-0.6427, -0.5332, -1.0963,  0.3655,  1.5181, -0.4637, -1.3757,  0.6313]],
       grad_fn=<SelectBackward>)

In [12]:
feature_mean

tensor([[0.1412, 0.1195, 0.1757, 0.1956, 0.2014, 0.1619, 0.1296, 0.1212, 0.1625,
         0.2444, 0.2788, 0.3305, 0.3097, 0.3634, 0.2017, 0.1454, 0.0949, 0.0646,
         0.0738, 0.0580, 0.0685, 0.0834, 0.1497, 0.3198, 0.2564, 0.2706, 0.2274,
         0.2668, 0.4305, 0.2778, 0.3106, 0.2719, 0.2167, 0.0923, 0.0908, 0.1187,
         0.3236, 0.6264, 0.3948, 0.4090, 0.3418, 0.3622, 0.3230, 0.1912, 0.2707,
         0.2215, 0.1806, 0.0852, 0.0931, 0.1090, 0.2908, 0.4428, 0.1921, 0.2183,
         0.2172, 0.3397, 0.3572, 0.2309, 0.2297, 0.1774, 0.2011, 0.1361, 0.1507,
         0.1538, 0.2483, 0.2350, 0.1305, 0.1665, 0.2978, 0.4957, 0.3393, 0.1802,
         0.1848, 0.1314, 0.2549, 0.3474, 0.3141, 0.2434, 0.2089, 0.1747, 0.2000,
         0.2210, 0.3853, 0.5794, 0.2008, 0.1367, 0.1783, 0.1296, 0.2880, 0.4784,
         0.5109, 0.4420, 0.3456, 0.3144, 0.3219, 0.3269, 0.3614, 0.5044, 0.2522,
         0.1661, 0.2265, 0.1435, 0.2713, 0.4259, 0.4878, 0.4517, 0.3450, 0.2899,
         0.2741, 0.2950, 0.3

In [13]:
# How to concatenate context and embedding word
torch.cat([embeddings[:, index], feature_mean], dim=1)

tensor([[-0.6427, -0.5332, -1.0963,  0.3655,  1.5181, -0.4637, -1.3757,  0.6313,
          0.1412,  0.1195,  0.1757,  0.1956,  0.2014,  0.1619,  0.1296,  0.1212,
          0.1625,  0.2444,  0.2788,  0.3305,  0.3097,  0.3634,  0.2017,  0.1454,
          0.0949,  0.0646,  0.0738,  0.0580,  0.0685,  0.0834,  0.1497,  0.3198,
          0.2564,  0.2706,  0.2274,  0.2668,  0.4305,  0.2778,  0.3106,  0.2719,
          0.2167,  0.0923,  0.0908,  0.1187,  0.3236,  0.6264,  0.3948,  0.4090,
          0.3418,  0.3622,  0.3230,  0.1912,  0.2707,  0.2215,  0.1806,  0.0852,
          0.0931,  0.1090,  0.2908,  0.4428,  0.1921,  0.2183,  0.2172,  0.3397,
          0.3572,  0.2309,  0.2297,  0.1774,  0.2011,  0.1361,  0.1507,  0.1538,
          0.2483,  0.2350,  0.1305,  0.1665,  0.2978,  0.4957,  0.3393,  0.1802,
          0.1848,  0.1314,  0.2549,  0.3474,  0.3141,  0.2434,  0.2089,  0.1747,
          0.2000,  0.2210,  0.3853,  0.5794,  0.2008,  0.1367,  0.1783,  0.1296,
          0.2880,  0.4784,  

In [14]:
predictions, attention_scores = decoder.forward(feature_maps, feature_mean, caption_batch)

In [15]:
caption_batch.shape

torch.Size([2, 51])

Number of predicted words == caption_len - 1

In [16]:
print(predictions.shape)

torch.Size([50, 2, 10004])


Random guess prediction probability

In [17]:
predictions[0, 0, :100]

tensor([ 0.6739,  0.3229,  0.0155,  0.6507, -1.5560,  1.2677,  1.1888, -0.3607,
        -0.1947,  0.6570,  0.6979, -0.8885, -0.1760,  0.0640, -1.0285, -0.1173,
        -0.0767,  1.0836,  0.3782, -0.5258, -0.1298, -0.2505, -0.4481, -0.0457,
        -0.3135, -0.1786,  0.9826,  0.1507, -0.9826, -0.2397,  1.6114,  1.0196,
         1.2116,  0.0103,  0.1678,  0.0821, -0.8459,  0.8452, -1.1444,  0.1135,
         0.1507,  0.2356,  0.2558,  0.3779, -0.3303, -0.2665,  0.9263,  1.0020,
         0.0422, -0.5091,  0.3531,  1.7150,  0.3366,  0.4141,  0.5544,  0.5730,
         1.1421, -0.5949, -0.2747,  0.4686,  0.4615,  0.1606, -0.6717,  0.8637,
         0.5027,  0.1546,  0.4419, -0.7613, -0.2036, -0.8299,  0.4595,  0.1669,
         0.2161, -0.3864, -0.9461, -0.4482, -0.8344,  0.3730, -0.8927, -0.8864,
        -0.3197,  0.8822,  0.4616, -0.2204, -0.5290, -0.0512,  0.4894, -1.0459,
         0.5827,  0.3709,  0.5624, -0.1604, -0.9301, -1.4108,  0.5452,  0.1847,
        -0.1478,  0.2389, -0.9372,  0.87

In [18]:
1 / 10_004

9.996001599360256e-05

Attention_scores

In [19]:
attention_scores.shape

torch.Size([50, 2, 512])

In [20]:
attention_scores[0, 0]

tensor([0.0018, 0.0027, 0.0019, 0.0033, 0.0017, 0.0017, 0.0022, 0.0017, 0.0016,
        0.0029, 0.0016, 0.0020, 0.0014, 0.0020, 0.0025, 0.0017, 0.0032, 0.0020,
        0.0017, 0.0017, 0.0022, 0.0018, 0.0027, 0.0016, 0.0018, 0.0018, 0.0018,
        0.0026, 0.0029, 0.0013, 0.0017, 0.0018, 0.0018, 0.0019, 0.0018, 0.0016,
        0.0026, 0.0017, 0.0017, 0.0018, 0.0017, 0.0031, 0.0018, 0.0016, 0.0020,
        0.0026, 0.0030, 0.0020, 0.0015, 0.0017, 0.0017, 0.0015, 0.0017, 0.0019,
        0.0018, 0.0017, 0.0018, 0.0016, 0.0018, 0.0018, 0.0017, 0.0017, 0.0021,
        0.0016, 0.0029, 0.0017, 0.0017, 0.0018, 0.0021, 0.0017, 0.0016, 0.0017,
        0.0018, 0.0023, 0.0018, 0.0046, 0.0013, 0.0020, 0.0025, 0.0017, 0.0017,
        0.0017, 0.0016, 0.0022, 0.0015, 0.0018, 0.0017, 0.0025, 0.0020, 0.0020,
        0.0021, 0.0017, 0.0025, 0.0034, 0.0025, 0.0017, 0.0018, 0.0019, 0.0020,
        0.0020, 0.0019, 0.0016, 0.0017, 0.0018, 0.0017, 0.0017, 0.0020, 0.0018,
        0.0026, 0.0014, 0.0017, 0.0017, 